# Modèle de détection de fake news
Ce Notebook se concentre sur la détection d'articles de fake news à l'aide de Mistral7B.

A la suite de l'utilisation de roBERTa, nous utilisons le LLM Mistral-7B-Instruct pour comparer les perfromances de ce modèle LLM plus récent. Cette tâche consiste à déterminer si le titre d'un article de presse est en accord, en désaccord, neutre ou discutable par rapport au contenu de l'article.

Le modèle Mistral-7B-Instruct, doté de 7 milliards de paramètres, a été spécifiquement affiné pour répondre à des instructions précises, ce qui le rend particulièrement adapté à notre projet de classification basé sur des prompts.

Étant donné la taille conséquente du modèle, une étape de quantification est nécessaire pour faciliter son exécution dans notre environnement de travail. Pour cela, nous utiliserons les outils suivants :

llama.cpp : un outil permettant de quantifier le modèle, réduisant ainsi l'empreinte mémoire et accélérant l'exécution.
llama-cpp-python : un wrapper Python pour llama.cpp, facilitant l'intégration de la quantification dans notre pipeline Python.
langchain : cette bibliothèque offre une interface simplifiée pour interagir avec des modèles de langage. Elle inclut la classe LlamaCpp, qui agit comme un wrapper autour de llama-cpp-python, permettant une utilisation intuitive du modèle LLM dans notre projet.


![Mistral7B](https://cdn.labellerr.com/1%20mistral%207b/SWA.webp)


 Le mécanisme de Sliding Windows Attention dans le modèle Mistral 7B permet d'étendre la portée d'attention du modèle au-delà d'une taille fixe. Cette technique divise la séquence d'entrée en segments plus petits, traités indépendamment, permettant ainsi au modèle de gérer efficacement jusqu'à 131 000 tokens. Cette approche améliore la génération de séquences en calculant l'attention à l'intérieur et au-delà de chaque segment, contribuant ainsi aux performances exceptionnelles du modèle Mistral 7B.








In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
!pip install pandas numpy transformers torch tqdm datasets trl accelerate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 10.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install peft bitsandbytes auto-gptq optimum trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.4/403.4 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
from datasets import Dataset

# Chargement des données
# train_stances = pd.read_csv('train_stances_with_summaries.csv')
# train_bodies = pd.read_csv('train_bodies_with_summaries.csv')
path_to_train_stances = '/content/drive/MyDrive/Data/train_stances_with_summaries.csv'
path_to_train_bodies = '/content/drive/MyDrive/Data/train_bodies_with_summaries.csv'

train_stances = pd.read_csv(path_to_train_stances)
train_bodies = pd.read_csv(path_to_train_bodies)
# Affichage des noms des colonnes pour comprendre la structure des données
print("Colonnes dans train_stances:", train_stances.columns.tolist())
print("Colonnes dans train_bodies:", train_bodies.columns.tolist())

# Vérification des premières lignes pour comprendre les données
print("Aperçu de train_stances:")
print(train_stances.head())
print(train_stances.info())
print("Aperçu de train_bodies:")
print(train_bodies.head())
print(train_bodies.info())

# Fusion des ensembles de données sur 'Body ID'
# Ceci permet de rassembler les informations des deux fichiers CSV
merged_df = pd.merge(
    train_stances[['Body ID', 'Headline', 'Stance']],
    train_bodies[['Body ID', 'articleBody']],
    on='Body ID',
    how='left'
)

# On renomme des colonnes pour plus de simplicité
merged_df.rename(columns={
    'Headline': 'headline',
    'articleBody': 'article_body',
    'Stance': 'stance'
}, inplace=True)

# Combinaison des textes pour la détection de position (stance)
merged_df['combined_text'] = merged_df.apply(
    lambda x: f"Titre: {x['headline']} \nArticle: {x['article_body']} \nPosition: {x['stance']}",
    axis=1
)

# Utilisation d'un échantillon du dataset pour l'entraînement nécessaire car lorsque j'ai chargé
# Avec tout le dataset le modèle ne pouvait pas être entraîné
# Donc on on sélectionne aléatoirement 1000 échantillons pour l'entraînement
data = Dataset.from_pandas(merged_df.sample(n=100, random_state=42))


Colonnes dans train_stances: ['Headline', 'Body ID', 'Stance', 'processed_headline', 'summary']
Colonnes dans train_bodies: ['Body ID', 'articleBody', 'processed_body', 'summary']
Aperçu de train_stances:
                                            Headline  Body ID     Stance  \
0  Police find mass graves with at least '15 bodi...      712  unrelated   
1  Hundreds of Palestinians flee floods in Gaza a...      158      agree   
2  Christian Bale passes on role of Steve Jobs, a...      137  unrelated   
3  HBO and Apple in Talks for $15/Month Apple TV ...     1034  unrelated   
4  Spider burrowed through tourist's stomach and ...     1923   disagree   

                                  processed_headline  \
0  police find mass graf least body near mexico t...   
1  hundred palestinian flee flood gaza israel ope...   
2  christian bale pass role steve job actor repor...   
3  hbo apple talk month apple tv streaming servic...   
4              spider burrowed tourist stomach chest   

 

In [11]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import torch

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Préparation des textes combinés et des étiquettes (labels)
combined_texts = merged_df['combined_text'].tolist()
stances = merged_df['stance'].tolist()

# Conversion des étiquettes textuelles en formes numériques
label_dict = {'unrelated': 0, 'agree': 1, 'disagree': 2, 'discuss': 3}
numeric_labels = [label_dict[stance] for stance in merged_df['stance']]


# Création d'un Dataset Hugging Face
hf_dataset = Dataset.from_pandas(merged_df)
hf_dataset = hf_dataset.map(lambda examples: tokenizer(examples['combined_text'], truncation=True, padding='max_length', max_length=512), batched=True)
hf_dataset = hf_dataset.map(lambda examples: {'labels': [label_dict[stance] for stance in examples['stance']]}, batched=True)

# Tokenisation
train_encodings = tokenizer(combined_texts, truncation=True, padding=True, max_length=512)
train_labels = torch.tensor(numeric_labels)

# Sauvegarde du tokenizer
tokenizer_save_path = "/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000"
tokenizer.save_pretrained(tokenizer_save_path)



Map:   0%|          | 0/49972 [00:00<?, ? examples/s]

Map:   0%|          | 0/49972 [00:00<?, ? examples/s]

('/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000/tokenizer_config.json',
 '/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000/special_tokens_map.json',
 '/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000/tokenizer.model',
 '/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000/added_tokens.json',
 '/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000/tokenizer.json')

In [6]:
import torch
from torch.utils.data import TensorDataset
from transformers import AutoTokenizer

# Chargement du tokeniser
tokenizer_save_path = "/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)


# Chargement des données de test
path_to_test_stances = '/content/drive/MyDrive/Data/competition_test_stances.csv'
test_stances = pd.read_csv(path_to_test_stances)
test_stances = test_stances[test_stances.apply(lambda x: len(x) == 3, axis=1)]

# Prétraitement des données de test
test_texts = test_stances['Headline'].copy()
test_texts.dropna(inplace=True)
test_texts = test_texts.astype(str)

# Tokenisation des données de test
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Conversion en format compatible avec PyTorch
test_dataset = TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)


In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from tqdm.auto import tqdm
import time


# Configuration de l'environnement CUDA
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:50"

def finetune_mistral_7b_stance_detection():
    start_time = time.time()  # Début du chronométrage

    # Configuration du modèle
    quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)

    bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
    )


    model = AutoModelForCausalLM.from_pretrained(
        "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
        quantization_config=quantization_config_loading,
        device_map="auto"
    )
    print(model)
    model.config.use_cache = False
    model.gradient_checkpointing_enable()
    model.config.pretraining_tp=1
    model = prepare_model_for_kbit_training(model)

    peft_config = LoraConfig(
        r=64,  # Tester avec 16
        lora_alpha=16,
        lora_dropout=0.1,  # Tester avec 0.05
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "v_proj"]
    )
    model = get_peft_model(model, peft_config)

    # Configuration de l'entraînement
    training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-stance-detection",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        optim="paged_adamw_8bit",
        learning_rate= 2e-4,
        weight_decay= 0.001,
        lr_scheduler_type="constant",
        num_train_epochs=1,
        fp16=True,
        max_grad_norm= 0.3,
        logging_steps=20,
        save_total_limit=2,
        warmup_ratio= 0.3
    )

    # Configuration du Trainer
    trainer = SFTTrainer(
        model=model,
        train_dataset=hf_dataset,
        args=training_arguments,
        tokenizer=tokenizer,
        dataset_text_field="combined_text",
        max_seq_length=512,
        packing=False
    )

    # Entraînement du modèle
    for epoch in tqdm(range(training_arguments.num_train_epochs), desc="Epochs d'entraînement"):
        trainer.train()
        torch.cuda.empty_cache()

    # Sauvegarde et partage du modèle
    model.save_pretrained("mistral-finetuned-stance-detection")

    end_time = time.time()  # Fin du chronométrage
    print(f"Entraînement terminé en {end_time - start_time:.2f} secondes")

if __name__ == "__main__":
    finetune_mistral_7b_stance_detection()

KeyboardInterrupt: ignored

In [7]:
# Chargement des données de test
test_stances = pd.read_csv('/content/drive/MyDrive/Data/competition_test_stances.csv')
test_bodies = pd.read_csv('/content/drive/MyDrive/Data/test_bodies_with_summaries.csv')

# Affichage des noms des colonnes pour comprendre la structure des données
print("Colonnes dans test_stances:", test_stances.columns.tolist())
print("Colonnes dans test_bodies:", test_bodies.columns.tolist())

# # Vérification des premières lignes pour comprendre les données
# print("Aperçu de test_stances:")
# print(test_stances.head())
# print(test_stances.info())
# print("Aperçu de test_bodies:")
# print(test_bodies.head())
# print(test_bodies.info())



# Fusion des ensembles de données sur 'Body ID'
# Ceci permet de rassembler les informations des deux fichiers CSV
test_merged = pd.merge(
    test_stances[['Body ID', 'Headline', 'Stance']],
    test_bodies[['Body ID', 'articleBody']],
    on='Body ID',
    how='left'
)

# Renommage des colonnes pour plus de clarté
test_merged.rename(columns={
    'Headline': 'headline',
    'articleBody': 'article_body',
    'Stance': 'stance'
}, inplace=True)

# Combinaison des textes pour la détection de position (stance)
test_merged['combined_text'] = test_merged.apply(
    lambda x: f"Titre: {x['headline']} \nArticle: {x['article_body']} \nPosition: {x['stance']}",
    axis=1
)



Colonnes dans test_stances: ['Headline', 'Body ID', 'Stance']
Colonnes dans test_bodies: ['Body ID', 'articleBody', 'processed_body', 'summary']


In [17]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, GPTQConfig
import torch
import time
import pandas as pd
from tqdm import tqdm

def stance_to_numeric(stance):
    # Définit une correspondance des étiquettes textuelles aux étiquettes numériques
    label_dict = {'unrelated': 0, 'agree': 1, 'disagree': 2, 'discuss': 3}
    return label_dict.get(stance, -1)

def inference_mistral_7b_stance_detection(batch_size=64):
    start_time = time.time()  # Commence le chronométrage

    # Chargement du modèle et du tokenizer
    model_path = "/content/drive/MyDrive/finetune_mistral_7b_stance_detection/checkpoint-3000"

    quantization_config = GPTQConfig(bits=4, disable_exllama=True)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_path,
        quantization_config=quantization_config
    ).to("cuda")

    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Chargement et préparation des données de test
    test_stances = pd.read_csv('/content/drive/MyDrive/Data/competition_test_stances.csv')
    test_bodies = pd.read_csv('/content/drive/MyDrive/Data/test_bodies_with_summaries.csv')
    test_merged = pd.merge(
        test_stances[['Body ID', 'Headline', 'Stance']],
        test_bodies[['Body ID', 'articleBody']],
        on='Body ID',
        how='left'
    )
    test_merged['combined_text'] = test_merged.apply(
        lambda x: f"Titre: {x['Headline']} \nArticle: {x['articleBody']} \nPosition: {x['Stance']}",
        axis=1
    )

    # Tokenisation des données de test
    test_encodings = tokenizer(test_merged['combined_text'].tolist(), truncation=True, padding=True, max_length=512, return_tensors="pt").to("cuda")

    # Génération des prédictions
    model.eval()
    predictions = []
    for i in tqdm(range(0, len(test_encodings['input_ids']), batch_size), desc="Processing Inference"):
        batch_input_ids = test_encodings['input_ids'][i:i+batch_size]
        batch_attention_mask = test_encodings['attention_mask'][i:i+batch_size]
        with torch.no_grad():
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            batch_predictions = torch.argmax(outputs.logits, dim=-1)
            predictions.extend(batch_predictions.cpu().numpy())


    # Décodage des prédictions
    decoded_predictions = np.array(predictions)

    # Stances actuelles
    actual_stances = test_merged['stance'].apply(lambda x: stance_to_numeric(x)).to_numpy()

    # Calcul des métriques
    accuracy = accuracy_score(actual_stances, decoded_predictions)
    f1 = f1_score(actual_stances, decoded_predictions, average='weighted')
    conf_matrix = confusion_matrix(actual_stances, decoded_predictions)

    print("Précision :", accuracy)
    print("Score F1 :", f1)
    print("Matrice de confusion :\n", conf_matrix)

    end_time = time.time()  # Fin du chronométrage
    print(f"Inference terminée en {end_time - start_time:.2f} secondes")

if __name__ == "__main__":
    inference_mistral_7b_stance_detection()


Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.1-GPTQ and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: ignored